In [1]:
## Imports

import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Github repo installation of paddle
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
# Install paddle OCR
!pip install paddleocr
# git clone
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
# cv2
!pip install "opencv-python-headless<4.3"


Looking in indexes: https://mirror.baidu.com/pypi/simple
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [2]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import os

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': Image.BICUBIC,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:39: DeprecationWarning: BOX is deprecated and will be removed in Pillo

In [3]:
ocr_model = PaddleOCR(lang='en')

def extract_text(image_path):
    result = ocr_model.ocr(image_path)
    
    return [res[1][0] for res in result]

[2022/09/02 09:32:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='/home/studio-lab-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='/home/studio-lab-user/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_bat

---    Fused 0 subgraphs into layer_norm op.
---    Fused 0 subgraphs into layer_norm op.


In [4]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("easy_results.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [52]:
%cd /home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images

/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images


In [53]:
image_dff = data[['image','relevant']]

In [54]:
image_dff

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0
465,a2db2c697881750d70feaa1a90d4f271338b9337,0
466,077d6694396e202064fb5b01b42837d2949e4ce7,0
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0


In [55]:
# # OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
# import pandas as pd
# import os
# image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [56]:
# image_dff
# image_dff.drop(index=0, inplace=True)

In [57]:
image_list = image_dff['image'].tolist()
len(image_list)

469

In [58]:
# text_list = []
# for image_path in image_list:
#     result = ocr_model.ocr(image_path)
#     text = [res[1][0] for res in result]
#     text_list.append(text)
# print(text_list)

In [59]:
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    
    image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    result = ocr_model.ocr(image_path)
    text = [res[1][0] for res in result]
    scores = [res[1][1] for res in result]
    result = [text, scores]
    return result

In [60]:
image_dff["words"] = image_dff.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_dff

[2022/09/02 10:32:01] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 10:32:01] ppocr DEBUG: dt_boxes num : 2, elapse : 0.544407844543457
[2022/09/02 10:32:01] ppocr DEBUG: rec_res num  : 2, elapse : 0.13804078102111816
[2022/09/02 10:32:01] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 10:32:02] ppocr DEBUG: dt_boxes num : 1, elapse : 0.7626700401306152
[2022/09/02 10:32:02] ppocr DEBUG: rec_res num  : 1, elapse : 0.1013944149017334
[2022/09/02 10:32:02] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 10:32:02] ppocr DEBUG: dt_boxes num : 0, elapse : 0.07672357559204102
[2022/09/02 10:32:02] ppocr DEBUG: rec_res num  : 0, elapse : 9.5367431640625e-07
[2022/09/02 10:32:02] ppocr WARNING: Since the angle classifier i

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[AMONGUS], [0.8279706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz.en.alibaba.com], [0.9213441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[Impostor], [0.9747896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[Press to make a sound], [0.9968701601028442]]"
...,...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0,"[[EMERGEWCN], [0.8580133318901062]]"
465,a2db2c697881750d70feaa1a90d4f271338b9337,0,"[[MW, 2022, 2022, 2022], [0.504313051700592, 0..."
466,077d6694396e202064fb5b01b42837d2949e4ce7,0,"[[SHHHHHHH!], [0.9825758337974548]]"
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0,"[[], []]"


In [61]:
import re
import string
def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [62]:
image_dff["words"] = image_dff.apply(clean_text, axis=1)

In [63]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [64]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [65]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score

In [69]:
import itertools
import nltk
import numpy as np

image_dff["OCR_score"] = image_dff.apply(get_final_score, axis=1)

In [70]:
image_dff = image_dff.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
image_dff

,image,relevant,words,OCR_score
277,4c6bc57673884a876168e5826912ac40d40e43b7,1,"[[impostor, vwomc n2, among us], [0.9978665709...",0.997867
49,c2d3fb2eca44bad695cd84e02c673cc27fdc175d,1,"[[impostor, vwomc n2, among us], [0.9973043203...",0.997304
229,e0614be54bfaf19965434fef71f85b18c9a46f1c,1,"[[impostor, vwonc n2, among us], [0.9966398477...",0.996640
227,3fe8539d691409d2207cf5f2bd5ce453c79fe185,1,"[[among us, impostor, among us, impostor], [0....",0.995405
53,75f467a04f86432f8e752744a486c415e7270c9b,1,"[[among us, impostor], [0.9149707555770874, 0....",0.995356
72,1a660508ae3183a16538facdcbebdb17428b2a4a,1,"[[piscuss , emergency, crewme, ieeting, impos,...",0.995231
438,8b3790e98660f3b1949f77ac42542169519f23c1,1,"[[impostor], [0.9952204823493958]]",0.995220
242,d8ed68c8657219f8e11febbf689adf8889e5a7ad,1,"[[impostor, vwona n2, among us], [0.9951813220...",0.995181
280,63094b2fcfe46bc394901b231968251c43b3e876,1,"[[impostor, vwona n2, among us], [0.9951813220...",0.995181
132,ae008c50c09aaee6babced0b3cd5b6551c32d82d,1,"[[impostor, vwona n2, among us], [0.9951813220...",0.995181


In [71]:
RA_score = roc_auc_score(image_dff["relevant"], image_dff["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

Rekognition ROC AUC Score =  0.91


In [73]:
image_dff.to_csv('/home/studio-lab-user/DS-Projects/OCR/paddle_ocr_v2.csv')

In [ ]:
# Reference https://www.youtube.com/watch?v=t5xwQguk9XU

In [74]:
# Dataframe of true positives

tp = image_dff.loc[(image_dff['relevant'] == 1) & (image_dff['OCR_score'] >= 0.5)] 
print(len(tp))

68


In [75]:
# Dataframe of true negatives

tn = image_dff.loc[(image_dff['relevant'] == 0) & (image_dff['OCR_score'] < 0.5)] 
print(len(tn))

318


In [76]:
# Dataframe of false positives

fp = image_dff.loc[(image_dff['relevant'] == 0) & (image_dff['OCR_score'] >= 0.5)] 
print(len(fp))

0


In [77]:
# Dataframe of false negatives

fn = image_dff.loc[(image_dff['relevant'] == 1) & (image_dff['OCR_score'] < 0.5)] 
print(len(fn))

83


In [78]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

1.0


In [79]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

0.4503311258278146


In [80]:
fn

,image,relevant,words,OCR_score
98,1f677c2708856bb52f9180481ff7c0b038d4bbe5,1,"[[monl vo, amongu, among us, among, aong us, s...",0.487474
182,bd06ecbf6df132526bc8950275b03d27e96e7764,1,"[[ppr, amon, ssns, ng us, sn, suss, v6l, repor...",0.428018
424,b8019c2cad1fb1584afefcb65b65b912725e011e,1,"[[sus , tor, amongus, you re, dumb], [0.8812...",0.373886
47,8f9b4aae8bab89538677a5c6f1290d89eaa8f5f2,1,"[[vwoud n2, among us, mp o stof], [0.664651274...",0.370264
413,8fc1d09c376fc80d7fd3ab840d70d001e5d22cdc,1,"[[shu, apmong us], [0.9938931465148926, 0.8950...",0.354170
421,588fbfc391a6644ba71888d68246af0a38ddff66,1,"[[present, rsloth, amongus, polus], [0.9907363...",0.349586
6,533a514c8c422f93173db706aeb8a2e87603e575,1,"[[among us, among, amonc us, you, you, look, l...",0.331106
325,a0a10018827f0156be2eca6e2b122eaaabf22a3e,1,"[[imong], [0.8793975710868835]]",0.329774
405,27892264f6744b357ae7bcba3dbf2e3beb05c29a,1,"[[i mvery good, at the hit game, among us, ify...",0.249129
171,74e3a541a1865af479a509cf5e795d7405417303,1,"[[vwonq n2, among us, amongus], [0.76226413249...",0.248194
